# Import Required Libraries
Import the necessary libraries, including OpenAI and Claude API clients.

In [ ]:
# Import Required Libraries
import openai
import requests
from dotenv import load_dotenv
import os

# Load API keys from .env file
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('CLAUDE_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')

# Initialize LLMs
Initialize the Google, OpenAI and Claude language models using their respective API keys.

In [ ]:
# Initialize LLMs

# Test OpenAI API call
def call_openai_api(prompt):
    response = openai.Completion.create(
        engine="davinci-codex",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Test Claude API call
def test_claude_api():
    prompt = "Hello, Claude!"
    response = call_claude_api(prompt)
    return response['choices'][0]['text'].strip()

# Test teh Gemini API call:
def test_gemini_api():
    prompt = "Hello, Gemini!"
    response = call_gemini_api(prompt)
    return response['choices'][0]['text'].strip()

# Test the API calls
openai_response = call_openai_api("Hello, OpenAI!")
claude_response = test_claude_api()
gemini_response = test_gemini_api()

openai_response, claude_response

In [ ]:
# Initialize Gemini, ChatGPT, and Sonnet

# Test Gemini API call
def call_gemini_api(prompt):
    response = requests.post(
        'https://api.gemini.ai/v1/completions',
        headers={'Authorization': f'Bearer {gemini_api_key}'},
        json={'prompt': prompt, 'max_tokens': 150}
    )
    return response.json()

# Test ChatGPT API call
def call_chatgpt_api(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Test Sonnet API call
def call_sonnet_api(prompt):
    response = requests.post(
        'https://api.sonnet.ai/v1/completions',
        headers={'Authorization': f'Bearer {sonnet_api_key}'},
        json={'prompt': prompt, 'max_tokens': 150}
    )
    return response.json()

# Test the API calls
gemini_response = call_gemini_api("Hello, Gemini!")
chatgpt_response = call_chatgpt_api("Hello, ChatGPT!")
sonnet_response = call_sonnet_api("Hello, Sonnet!")

print(gemini_response, chatgpt_response, sonnet_response)

# Define Interaction Variable
Define a variable to limit the number of interactions between the two models.

In [ ]:
# Define Interaction Variable
interaction_limit = 10  # Set the number of interactions between the two models

# Create Language Game
Define a function where one model creates a language game.

In [ ]:
# Create Language Game
def create_language_game(prompt):
    # OpenAI creates the initial language game
    openai_game = call_openai_api(prompt)
    
    # Claude evaluates the game
    claude_feedback = call_claude_api(f"Evaluate this game: {openai_game}")
    
    return openai_game, claude_feedback

# Example usage
initial_prompt = "Create a fun word association game."
game, feedback = create_language_game(initial_prompt)

game, feedback

# Evaluate Game
Define a function where the other model evaluates the game as fun or not and provides feedback.

In [ ]:
# Evaluate Game
def evaluate_game(game, feedback_prompt):
    # Claude evaluates the game
    claude_feedback = call_claude_api(feedback_prompt.format(game=game))
    
    # OpenAI provides feedback on Claude's evaluation
    openai_feedback = call_openai_api(f"Evaluate this feedback: {claude_feedback['choices'][0]['text'].strip()}")
    
    return claude_feedback['choices'][0]['text'].strip(), openai_feedback

# Example usage
game = "A word association game where players take turns saying words that are related to the previous word."
feedback_prompt = "Evaluate this game: {game}"
claude_feedback, openai_feedback = evaluate_game(game, feedback_prompt)

claude_feedback, openai_feedback

# Iterate Game Creation and Evaluation
Implement a loop to alternate between game creation and evaluation based on the interaction variable.

In [ ]:
# Iterate Game Creation and Evaluation
def iterate_game_creation_and_evaluation(initial_prompt, interaction_limit):
    current_prompt = initial_prompt
    for i in range(interaction_limit):
        # OpenAI creates the game
        openai_game = call_openai_api(current_prompt)
        
        # Claude evaluates the game
        claude_feedback = call_claude_api(f"Evaluate this game: {openai_game}")
        
        # OpenAI provides feedback on Claude's evaluation
        openai_feedback = call_openai_api(f"Evaluate this feedback: {claude_feedback['choices'][0]['text'].strip()}")
        
        # Update the prompt for the next iteration
        current_prompt = f"Improve this game based on feedback: {openai_feedback}"
        
        # Print the results of the current iteration
        print(f"Iteration {i+1}:")
        print(f"Game: {openai_game}")
        print(f"Claude's Feedback: {claude_feedback['choices'][0]['text'].strip()}")
        print(f"OpenAI's Feedback: {openai_feedback}")
        print("\n")

# Example usage
initial_prompt = "Create a fun word association game."
iterate_game_creation_and_evaluation(initial_prompt, interaction_limit)

In [ ]:
# Import Required Libraries
import openai
import requests

# Initialize LLMs
openai.api_key = 'your_openai_api_key'
claude_api_key = 'your_claude_api_key'
gemini_api_key = 'your_gemini_api_key'

# Define Interaction Variable
interaction_limit = 10

# User-defined Variables
thing_to_create = "language game"  # Change this to "machine learning algorithm", "marketing campaign strategy", etc.
evaluation_criteria = "fun"  # Change this to "usefulness", "effectiveness", "uniqueness", etc.

# Initialize Feedback Variables
feedback_buffer = []
collective_feedback = []

# Function to Create Thing
def create_thing(model, feedback):
    prompt = f"Create a {thing_to_create} based on the following feedback: {feedback}"
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=150)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 150})
        return response.json()['choices'][0]['text'].strip()

# Function to Evaluate Thing
def evaluate_thing(model, thing):
    prompt = f"Evaluate the following {thing_to_create} for {evaluation_criteria}: {thing}"
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=50)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 50})
        return response.json()['choices'][0]['text'].strip()

# Function to Evaluate Feedback using Google Gemini
def evaluate_feedback(feedback):
    prompt = f"Evaluate the usefulness of the following feedback for creating a good {thing_to_create}: {feedback}"
    response = requests.post('https://api.gemini.ai/v1/completions', headers={'Authorization': f'Bearer {gemini_api_key}'}, json={'prompt': prompt, 'max_tokens': 50})
    return response.json()['choices'][0]['text'].strip()

# Main Interaction Loop
for i in range(interaction_limit):
    # Create Thing
    if i % 2 == 0:
        thing = create_thing('openai', collective_feedback)
    else:
        thing = create_thing('claude', collective_feedback)
    
    # Evaluate Thing
    if i % 2 == 0:
        feedback = evaluate_thing('claude', thing)
    else:
        feedback = evaluate_thing('openai', thing)
    
    # Add Feedback to Buffer
    feedback_buffer.append(feedback)
    
    # Evaluate Feedback Buffer
    useful_feedback = []
    for fb in feedback_buffer:
        evaluation = evaluate_feedback(fb)
        if "useful" in evaluation:
            useful_feedback.append(fb)
    
    # Update Collective Feedback
    collective_feedback = useful_feedback

    # Print Current Thing and Feedback
    print(f"Round {i+1}")
    print(f"{thing_to_create.capitalize()}: {thing}")
    print(f"Feedback: {feedback}")
    print(f"Collective Feedback: {collective_feedback}")
    print("\n")

In [ ]:
# Import Required Libraries
import openai
import requests
from dotenv import load_dotenv
import os

# Load API keys from .env file
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('CLAUDE_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')

# Define Interaction Variable
interaction_limit = 10

# User-defined Variables
thing_to_create = "language game"  # Change this to "machine learning algorithm", "marketing campaign strategy", etc.
evaluation_criteria = "fun"  # Change this to "usefulness", "effectiveness", "uniqueness", etc.

# Initialize Feedback Variables
feedback_buffer = []
collective_feedback = []

# Function to Create Thing
def create_thing(model, feedback):
    prompt = f"Create a {thing_to_create} based on the following feedback: {feedback}"
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=150)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 150})
        return response.json()['choices'][0]['text'].strip()

# Function to Evaluate Thing
def evaluate_thing(model, thing):
    prompt = f"Evaluate the following {thing_to_create} for {evaluation_criteria}: {thing}"
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=50)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 50})
        return response.json()['choices'][0]['text'].strip()

# Function to Evaluate Feedback using Google Gemini
def evaluate_feedback(feedback):
    prompt = f"Evaluate the usefulness of the following feedback for creating a good {thing_to_create}: {feedback}"
    response = requests.post('https://api.gemini.ai/v1/completions', headers={'Authorization': f'Bearer {gemini_api_key}'}, json={'prompt': prompt, 'max_tokens': 50})
    return response.json()['choices'][0]['text'].strip()

# Main Interaction Loop
for i in range(interaction_limit):
    # Create Thing
    if i % 2 == 0:
        thing = create_thing('openai', collective_feedback)
    else:
        thing = create_thing('claude', collective_feedback)
    
    # Evaluate Thing
    if i % 2 == 0:
        feedback = evaluate_thing('claude', thing)
    else:
        feedback = evaluate_thing('openai', thing)
    
    # Add Feedback to Buffer
    feedback_buffer.append(feedback)
    
    # Evaluate Feedback Buffer
    useful_feedback = []
    for fb in feedback_buffer:
        evaluation = evaluate_feedback(fb)
        if "useful" in evaluation:
            useful_feedback.append(fb)
    
    # Update Collective Feedback
    collective_feedback = useful_feedback

    # Print Current Thing and Feedback
    print(f"Round {i+1}")
    print(f"{thing_to_create.capitalize()}: {thing}")
    print(f"Feedback: {feedback}")
    print(f"Collective Feedback: {collective_feedback}")
    print("\n")